In [1]:
# Importacion de recusos generales
import joblib
import pandas as pd
import numpy as np
import sqlite3
from config.parametros import RutaDataSet as RT
# Recursos propios
from services.transformaciones import CargaDatos
# Recursos de visualizacion
import matplotlib.pyplot as plt
import seaborn as sns
# Preprocesamiento
from sklearn.preprocessing import MinMaxScaler
import joblib
from services.transformaciones import CargaDatos
from config.parametros import RutaDataSet as RT
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

In [2]:
# Instancia de conexion a la base de datos SQL
conn = sqlite3.connect(RT.DataBase)
cursor = conn.cursor()

In [3]:
# Join de la tabla Qualitative y Campanas
query = """
    SELECT
        c.ID,
        c.mora,
        c.Vivienda,
        c.Consumo,
        c.Mes,
        c.Dias_Ultima_Camp,
        c.No_Contactos,
        c.Resultado_Anterior,
        c.emp_var_rate,
        c.cons_price_idx,
        c.cons_conf_idx,
        c.euribor3m,
        c.nr_employed,
        c.y,
        q.Edad,
        q.Tipo_Trabajo,
        q.Estado_Civil,
        q.Educacion,
        q.Contacto
    FROM Campanas AS c
    JOIN Qualitative AS q ON c.ID = q.ID
    WHERE y IS NULL
"""
df = pd.read_sql_query(query,conn)

In [4]:
# Definicion de variables
list_min_max = [
    'Dias_Ultima_Camp',
    'No_Contactos',
    'emp_var_rate',
    'cons_price_idx',
    'cons_conf_idx',
    'euribor3m',
    'nr_employed',
    'Edad'
]
list_get_dummies = [
    'mora',
    'Vivienda',
    'Consumo',
    'Mes',
    'Resultado_Anterior',
    'Tipo_Trabajo',
    'Estado_Civil',
    'Educacion',
    'Contacto'
]

In [5]:
# Normalizacion de Dataset en columnas especificadas en list_min_max
scaler = MinMaxScaler()
df[list_min_max] = scaler.fit_transform(df[list_min_max])

In [6]:
# Generar columnas usando metodo one-hot encoding a columnas detalladas en list_get_dummies
df_matriz = pd.get_dummies(df,columns=list_get_dummies)
col_bool = df_matriz.iloc[:,-50:]
df_matriz.iloc[:,-50:] = col_bool.astype(int)

C:\Users\bi\AppData\Local\Temp\ipykernel_13168\3671179923.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       1
1       1
2       1
3       1
4       1
       ..
6924    1
6925    1
6926    1
6927    1
6928    1
Name: mora_no, Length: 6929, dtype: int32' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_matriz.iloc[:,-50:] = col_bool.astype(int)
C:\Users\bi\AppData\Local\Temp\ipykernel_13168\3671179923.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       0
1       0
2       0
3       0
4       0
       ..
6924    0
6925    0
6926    0
6927    0
6928    0
Name: mora_si, Length: 6929, dtype: int32' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_matriz.iloc[:,-50:] = col_bool.astype(int)
C:\Users\bi\AppData\Local\Temp\ipykernel_13168\3671179923.py:4

In [7]:
# Defincion de variables (x) y (y)
X_matriz = df_matriz.drop(columns=['ID','y'],axis=1)
y_matriz = df_matriz['y']
e_matriz = df_matriz['ID']
# Generarar Matricez vectoriales
X = X_matriz.to_numpy() # matriz con las variables independientes
y = y_matriz.to_numpy() # Matriz con la variables dependiente
e = e_matriz.to_numpy() # Vector on los ID de cliente

In [8]:
# Guardar Matriz para modelado
M = CargaDatos(carpeta='Archivos')
M.guardar_matriz_numpy(nombre_matriz='X_CDT',vectores=X)
M.guardar_matriz_numpy(nombre_matriz='y_CDT',vectores=y)
M.guardar_matriz_numpy(nombre_matriz='Etiqueta_cliente',vectores=e)

Matriz guardada exitosamente como X_CDT.npy
Matriz guardada exitosamente como y_CDT.npy
Matriz guardada exitosamente como Etiqueta_cliente.npy


In [9]:
# Importar la matriz X
X_matriz = CargaDatos('Archivos')
X = X_matriz.leer_matriz_numpy(nombre_matriz=RT.X)
X.shape

(6929, 58)

In [10]:
# Importar la matriz y
y_matriz = CargaDatos('Archivos')
y = y_matriz.leer_matriz_numpy(nombre_matriz=RT.y)
y.shape

(6929,)

In [11]:
# Importar la matriz e que contiene las etiquetas de la matriz
e_matriz = CargaDatos('Archivos')
e = e_matriz.leer_matriz_numpy(nombre_matriz=RT.e)
e.shape

(6929,)

In [12]:
# Crear y entrenar el modelo
model = joblib.load('Modelos_Regresion_Logistica.pkl')

# Hacer predicciones
y_pred = model.predict(X)

In [13]:
import pandas as pd

assert len(e) == len(y_pred)

# Crear un DataFrame con los IDs y las predicciones
resultados = pd.DataFrame({
    'ID': e.flatten(),  # Si e es un array o DataFrame, asegúrate de que sea un array unidimensional
    'y': y_pred
})

# Guardar el DataFrame en un archivo CSV
resultados.to_csv('resultado_predictions.csv', encoding='utf-8', sep=',', index=False)

# Mostrar las primeras filas del DataFrame de resultados
save = 'Predicciones.csv'
resultados.to_csv(save,encoding='utf-8',sep=',',index=False)
